In [ ]:
# Week 3: part 1 (#1-7) 
#         part 2 (#8-10)
#         part 3 (#11-last )
# Build a Toronto dataframe (postal code,neighborhood,borough name).
# Get the geographical coordinates of the neighborhoods.
# Explore and cluster the neighborhoods in Toronto

In [ ]:
#1.Import libraries
import requests
import urllib.request
from bs4 import BeautifulSoup
import numpy as np
from urllib.request import urlopen
import csv
import json
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 

In [ ]:
import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [ ]:
#2. Scrap data from wikipedia to dataframe
#GET request & parse html to beautiful soup object
wiki_url ='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(wiki_url).text
soup = BeautifulSoup(source,'lxml')

In [ ]:
#create lists for table data
postalcodelist = []
boroughlist = []
neighborhoodlist =[]

In [ ]:
#using BeautifulSoup find table rows
soup.find('table').find_all('tr')
row = soup.find('table').find_all('tr')

In [ ]:
#4. replace cells which Borough not assigned
#   append the data into the respective lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        if(cells[1].text.replace('\n','') != 'Not assigned') : 
            postalcodelist.append(cells[0].text.replace('\n',''))
            boroughlist.append(cells[1].text.strip().replace('\n',''))
            neighborhoodlist.append(cells[2].text.replace('\n','').replace("/", ","))
data_df.head() 

In [ ]:
# create a new DataFrame from the three lists
data_df = pd.DataFrame({"PostalCode": postalcodelist,
                        "Borough": boroughlist,
                        "Neighborhood": neighborhoodlist})

In [ ]:
# group neighborhoods in the same borough
data_df_group = data_df.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
data_df_group.head()

In [31]:
#5. for Neighborhood="Not assigned", turn the value the same as Borough
for index, row in data_df_group.iterrows():
    if row["Neighborhood"] == "Not assigned":
       row["Neighborhood"] = row["Borough"]
data_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [32]:
6.# Checking requirement fulfilled by creating a new test dataframe
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(data_df_group[data_df_group["PostalCode"]==postcode], ignore_index=True)   
    
test_df

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Parkview Hill , Woodbine Gardens"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Wexford , Maryvale"
7,M9V,Etobicoke,"South Steeles , Silverstone , Humbergate , Jam..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower , King and Spadina , Railway Lands , ..."


In [33]:
#7.print rows numbers of cleaned dataframe
data_df_group.shape

(103, 3)

In [34]:
#8. Loading the coordinates csv file from coursera
coordinates = pd.read_csv("Geospatial_Coordinates.csv")
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [35]:
# change name from "Postal Code" to  "PostalCode" column
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [36]:
#9. Merging two tables to get coordinates on "Postalcode" column
data_df_new = data_df_group.merge(coordinates, on="PostalCode", how="left")
data_df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [37]:
# 10. Checking requirement fulfilled by creating a new test dataframe
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(data_df_new[data_df_new["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Wexford , Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"South Steeles , Silverstone , Humbergate , Jam...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower , King and Spadina , Railway Lands , ...",43.628947,-79.394420


In [38]:
# 11. Use geopy library to obtain Toronto's latitude and longitude.
from geopy.geocoders import Nominatim
import folium

address = 'Toronto'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
DisplayMap_folium = folium.Map(location=[latitude, longitude], zoom_start=10)
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [24]:
# 12. Create Toronto Map 
# create Toronto Map using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(data_df_new['Latitude'], data_df_new['Longitude'], data_df_new['Borough'], data_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#000fff',
        fill_opacity=0.5).add_to(map_toronto)  
    
map_toronto

In [39]:
#13. Filter only boroughs that contain the word Toronto
borough_names = list(data_df_new.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

In [40]:
# create a new DataFrame with only boroughs that contain the word Toronto
data_df_new = data_df_new[data_df_new['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(data_df_new.shape)
data_df_new.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West , Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar , The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [27]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(data_df_new['Latitude'], data_df_new['Longitude'], data_df_new['Borough'], data_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [51]:
#14. Using Foursquare API,exploring the Neighbourhoods
# define Foursquare Credentials and Version
CLIENT_ID = '' # Foursquare ID
CLIENT_SECRET = ' ' # your Foursquare Secret
VERSION = '20200408' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET: 


In [49]:
#Get the top 100 venues that are within a radius of 500 meters.
radius = 200
LIMIT = 90

venues = []

for lat, long, post, borough, neighborhood in zip(data_df_new['Latitude'], data_df_new['Longitude'], data_df_new['PostalCode'], data_df_new['Borough'], data_df_new['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [50]:
# convert venues list to DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
)
venues_df.head()

SyntaxError: invalid syntax (<ipython-input-50-4c9e7ba2eafd>, line 6)

In [ ]:
#checking venues for each PostalCode
venues_df.groupby(["PostalCode", "Borough", "Neighborhood"]).count()

In [ ]:
#check how many unique categories
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

In [ ]:
venues_df['VenueCategory'].unique()[:50]

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

In [ ]:
toronto_group = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_group.shape)
toronto_group

In [ ]:
#finding top 10 venues for each Postalcode
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_sorted = pd.DataFrame(columns=columns)
neighborhoods_sorted['PostalCode'] = toronto_group['PostalCode']
neighborhoods_sorted['Borough'] = toronto_group['Borough']
neighborhoods_sorted['Neighborhoods'] = toronto_group['Neighborhoods']

for ind in np.arange(toronto_group.shape[0]):
    row_categories = toronto_group.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_sorted.shape)
neighborhoods_sorted